In [1]:
%matplotlib widget

In [130]:
# Imports
import sys
import os 
import numpy as np
import pandas as pd
from astropy.visualization import hist
from tqdm import tqdm
from scipy import constants
from scipy.stats import ks_2samp, anderson_ksamp

from spaxelsleuth.loaddata.lzifu import load_lzifu_galaxies
from spaxelsleuth.loaddata.sami import load_sami_galaxies
from spaxelsleuth.plotting.plottools import plot_empty_BPT_diagram
from spaxelsleuth.plotting.plottools import vmin_fn, vmax_fn, label_fn, cmap_fn, fname_fn
from spaxelsleuth.plotting.plottools import bpt_colours, bpt_labels, whav_colors, whav_labels
from spaxelsleuth.plotting.plottools import morph_labels, morph_ticks
from spaxelsleuth.plotting.plottools import ncomponents_labels, ncomponents_colours
from spaxelsleuth.plotting.plottools import component_labels, component_colours
from spaxelsleuth.plotting.plotgalaxies import plot2dhistcontours, plot2dscatter, plot2dcontours
from spaxelsleuth.plotting.plot2dmap import plot2dmap
from spaxelsleuth.plotting.sdssimg import plot_sdss_image

import matplotlib
from matplotlib import rc, rcParams
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from IPython.core.debugger import Tracer

rc("text", usetex=False)
rc("font",**{"family": "serif", "size": 11})
rcParams["savefig.bbox"] = "tight"
rcParams["savefig.format"] = "pdf"
plt.ion()
plt.close("all")


In [131]:
# Options
fig_path = "/priv/meggs3/u5708159/SAMI/figs/paper/"
savefigs = True
bin_type = "default"    # Options: "default" or "adaptive" for Voronoi binning
ncomponents = "recom"   # Options: "1" or "recom"
eline_SNR_min = 5       # Minimum S/N of emission lines to accept
plt.close("all")


In [4]:
# Load the sample
df = load_sami_galaxies(ncomponents=ncomponents,
                        bin_type=bin_type,
                        eline_SNR_min=eline_SNR_min, 
                        vgrad_cut=False,
                        line_amplitude_SNR_cut=True,
                        correct_extinction=False,
                        sigma_gas_SNR_cut=True)


/pkg/linux/anaconda/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/pkg/linux/anaconda/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/pkg/linux/anaconda/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

# Using the KS and AD 2-sample tests to investigate the drivers of the number of components measured in each spaxel
---

See [here](https://stats.stackexchange.com/questions/465196/kolmogorov-smirnov-test-statistic-interpretation-with-large-samples) for a good discussion.

Interpreting the statistic values: in both the KS and AD tests, larger values imply greater deviation between the underlying distributions of the two samples.

### Select subsample

In [125]:
# Full sample
df_1comp = df[df["Number of components"] == 1]
df_2comp = df[df["Number of components"] == 2]
df_3comp = df[df["Number of components"] == 3]
    
fname = os.path.join(fig_path, "hist_statistics_whole_sample.pdf")
fig_title = "All spaxels"

Number of 1-component spaxels: 608641
Number of 2-component spaxels: 44943
Number of 3-component spaxels: 4744


In [ ]:
# SF-only
df_SF = df.copy()
df_SF = df_SF[df_SF["BPT (total)"] == "SF"]

# Split by number of components
df_1comp = df_SF[df_SF["Number of components"] == 1]
df_2comp = df_SF[df_SF["Number of components"] == 2]
df_3comp = df_SF[df_SF["Number of components"] == 3]

fname = os.path.join(fig_path, "hist_statistics_SF_only.pdf")
fig_title = "SF spaxels only"

In [149]:
# SF-only, with beam smearing cut
df_SF_beam_smearing_cut = df_SF.copy()
for ii in range(3):
    cond_beam_smearing = df_SF_beam_smearing_cut[f"Beam smearing flag (component {ii})"] == True

    # NaN out offending cells
    df_SF_beam_smearing_cut.loc[cond_beam_smearing, 
           [f"v_gas (component {ii})",
            f"sigma_gas (component {ii})", 
            f"v_gas error (component {ii})",
            f"sigma_gas error (component {ii})",]] = np.nan

# Split by number of components
df_1comp = df_SF_beam_smearing_cut[df_SF_beam_smearing_cut["Number of components"] == 1]
df_2comp = df_SF_beam_smearing_cut[df_SF_beam_smearing_cut["Number of components"] == 2]
df_3comp = df_SF_beam_smearing_cut[df_SF_beam_smearing_cut["Number of components"] == 3]

fname = os.path.join(fig_path, "hist_statistics_SF_only_BScut.pdf")
fig_title = "SF spaxels only, beam smearing cut"

In [ ]:
# SF-only, with beam smearing cut, with spaxels at small radii removed
df_SF_large_radii = df_SF_beam_smearing_cut.copy()
df_SF_large_radii = df_SF_large_radii[df_SF_large_radii["r/R_e"] > 1.0]

# Split by number of components
df_1comp = df_SF_large_radii[df_SF_large_radii["Number of components"] == 1]
df_2comp = df_SF_large_radii[df_SF_large_radii["Number of components"] == 2]
df_3comp = df_SF_large_radii[df_SF_large_radii["Number of components"] == 3]

fname = os.path.join(fig_path, "hist_statistics_SF_only_BScut_large_radii.pdf")
fig_title = r"SF spaxels only, beam smearing cut, $r/R_e > 1$"

In [ ]:
# SF-only, with beam smearing cut, with spaxels at small radii removed, at low inclination
df_SF_large_radii = df_SF_beam_smearing_cut.copy()
df_SF_large_radii = df_SF_large_radii[df_SF_large_radii["r/R_e"] > 1.0]
df_SF_large_radii = df_SF_large_radii[df_SF_large_radii["Inclination i (degrees)"] < 30]
# df_SF_large_radii = df_SF_large_radii[df_SF_large_radii["z_spec"] < 0.04]

# Split by number of components
df_1comp = df_SF_large_radii[df_SF_large_radii["Number of components"] == 1]
df_2comp = df_SF_large_radii[df_SF_large_radii["Number of components"] == 2]
df_3comp = df_SF_large_radii[df_SF_large_radii["Number of components"] == 3]

fname = os.path.join(fig_path, "hist_statistics_SF_only_BScut_large_radii_low_inclination.pdf")
fig_title = r"SF spaxels only, beam smearing cut, $r/R_e > 1$, $i < 30^\circ$"

## Run KS, AD tests

In [ ]:
# Print statistics
for ii, df_comp in enumerate([df_1comp, df_2comp, df_3comp]):
    print(f"Number of {ii + 1}-component spaxels: {df_comp.shape[0]}")


In [135]:
alpha = 0.01  # p-value below which we reject the null hypothesis
cols = ["log HALPHA EW (total)", "log SFR (component 0)", "log SFR surface density (component 0)", "HALPHA extinction correction", 
        "sigma_gas (component 0)", "v_gas (component 0)", "sigma_gas - sigma_* (component 0)", "v_gas - v_* (component 0)", 
        "r/R_e", "log HALPHA EW (component 0)", "v_*", "sigma_*",
        "D4000", "mstar", "R_e (kpc)", "log(M/R_e)", 
        "Inclination i (degrees)", "Bin size (square kpc)", "z_spec", "v_grad (component 0)",]

fig, axs = plt.subplots(nrows=len(cols) // 4, ncols=4, figsize=(16, 20))
fig.subplots_adjust(wspace=0.35, hspace=0.4)
for col_x, ax in zip(cols, axs.flat):
    # Extract values
    d1 = df_1comp[col_x].dropna().values
    d2 = df_2comp[col_x].dropna().values
    
    # Run KS test 
    r_KS = ks_2samp(d1, d2)
    if r_KS.pvalue < alpha:
        print(f"KS test: {col_x}: the two distributions are different at a {alpha * 100:.3f}% level (p-value = {r_KS.pvalue * 100:.10f}%)")
        ax.text(s=r"KS: $%.2f$ ($p = %.3f$)" % (r_KS.statistic, r_KS.pvalue), x=0.05, y=0.95, verticalalignment="top", fontsize="small", transform=ax.transAxes, color="red")
    else:
        print(f"KS test: {col_x}: the two distributions are the same at a {alpha * 100:.3f}% level (p-value = {r_KS.pvalue * 100:.10f}%)")
        ax.text(s=r"KS: $%.2f$ ($p = %.3f$)" % (r_KS.statistic, r_KS.pvalue), x=0.05, y=0.95, verticalalignment="top", fontsize="small", transform=ax.transAxes)

    # Run the 2-sample KS test 
    r_AD = anderson_ksamp([d1, d2])
    if r_AD.significance_level < alpha:
        print(f"AD test: {col_x}: the two distributions are different at a {alpha * 100:.3f}% level (p-value = {r_AD.significance_level * 100:.5f}%)")
        ax.text(s=r"AD: $%.2f$ ($p \leq %.3f$)" % (r_AD.statistic, r_AD.significance_level), x=0.05, y=0.88, verticalalignment="top", fontsize="small", transform=ax.transAxes, color="red")
    else:
        print(f"AD test: {col_x}: the two distributions are the same at a {alpha * 100:.3f}% level (p-value = {r_AD.significance_level * 100:.5f}%)")
        ax.text(s=r"AD: $%.2f$ ($p = %.3f$)" % (r_AD.statistic, r_AD.significance_level), x=0.05, y=0.88, verticalalignment="top", fontsize="small", transform=ax.transAxes)
        
    # Plot the distributions of the quantity in 1, 2 and 3-component spaxels
    for ii, d in enumerate([d1, d2]):
        hist(d, density=True, histtype="step",
             ax=ax, 
             range=(vmin_fn(col_x), vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[ii + 1]} component{'s' if ii >= 1 else ''}" + r" ($N = %d$)" % len(d),
             color=ncomponents_colours[ii + 1])
    
    # Decorations
    ax.set_xlabel(label_fn(col_x) + " (component 1)" if "(component 0)" in col_x else label_fn(col_x))
    ax.autoscale(enable=True, axis="x", tight=True)
    ax.set_ylabel(r"$N$ (normalised)")
axs[0][0].legend(loc="lower center", bbox_to_anchor=[0.5, 1.1], fontsize="small")
fig.suptitle(fig_title)

# Save
if savefigs:
    print(f"Saving file to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")
    
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KS test: log HALPHA EW (total): the two distributions are different at a 1.000% level (p-value = 0.0000000000%)
AD test: log HALPHA EW (total): the two distributions are different at a 1.000% level (p-value = 0.10000%)
KS test: log SFR (component 0): the two distributions are different at a 1.000% level (p-value = 0.0000000000%)
AD test: log SFR (component 0): the two distributions are different at a 1.000% level (p-value = 0.10000%)


<ipython-input-135-275af09cda96>:25: UserWarning: p-value floored: true value smaller than 0.001
  r_AD = anderson_ksamp([d1, d2])
<ipython-input-135-275af09cda96>:25: UserWarning: p-value floored: true value smaller than 0.001
  r_AD = anderson_ksamp([d1, d2])


KS test: log SFR surface density (component 0): the two distributions are different at a 1.000% level (p-value = 0.0000000000%)
AD test: log SFR surface density (component 0): the two distributions are different at a 1.000% level (p-value = 0.10000%)
KS test: HALPHA extinction correction: the two distributions are different at a 1.000% level (p-value = 0.0000000000%)
AD test: HALPHA extinction correction: the two distributions are different at a 1.000% level (p-value = 0.10000%)


<ipython-input-135-275af09cda96>:25: UserWarning: p-value floored: true value smaller than 0.001
  r_AD = anderson_ksamp([d1, d2])
<ipython-input-135-275af09cda96>:25: UserWarning: p-value floored: true value smaller than 0.001
  r_AD = anderson_ksamp([d1, d2])


...

AD test: sigma_*: the two distributions are different at a 1.000% level (p-value = 0.10000%)
KS test: D4000: the two distributions are different at a 1.000% level (p-value = 0.0000000000%)
AD test: D4000: the two distributions are different at a 1.000% level (p-value = 0.10000%)
KS test: mstar: the two distributions are different at a 1.000% level (p-value = 0.0000000000%)
AD test: mstar: the two distributions are different at a 1.000% level (p-value = 0.10000%)


<ipython-input-135-275af09cda96>:25: UserWarning: p-value floored: true value smaller than 0.001
  r_AD = anderson_ksamp([d1, d2])
<ipython-input-135-275af09cda96>:25: UserWarning: p-value floored: true value smaller than 0.001
  r_AD = anderson_ksamp([d1, d2])
<ipython-input-135-275af09cda96>:25: UserWarning: p-value floored: true value smaller than 0.001
  r_AD = anderson_ksamp([d1, d2])
<ipython-input-135-275af09cda96>:25: UserWarning: p-value floored: true value smaller than 0.001
  r_AD = anderson_ksamp([d1, d2])
<ipython-input-135-275af09cda96>:25: UserWarning: p-value floored: true value smaller than 0.001
  r_AD = anderson_ksamp([d1, d2])


KS test: R_e (kpc): the two distributions are different at a 1.000% level (p-value = 0.0000000000%)
AD test: R_e (kpc): the two distributions are different at a 1.000% level (p-value = 0.10000%)
KS test: log(M/R_e): the two distributions are different at a 1.000% level (p-value = 0.0000000000%)
AD test: log(M/R_e): the two distributions are different at a 1.000% level (p-value = 0.10000%)
KS test: Inclination i (degrees): the two distributions are different at a 1.000% level (p-value = 0.0000000000%)
AD test: Inclination i (degrees): the two distributions are different at a 1.000% level (p-value = 0.10000%)
KS test: Bin size (square kpc): the two distributions are different at a 1.000% level (p-value = 0.0000000000%)
AD test: Bin size (square kpc): the two distributions are different at a 1.000% level (p-value = 0.10000%)
KS test: z_spec: the two distributions are different at a 1.000% level (p-value = 0.0000000000%)
AD test: z_spec: the two distributions are different at a 1.000% leve

<ipython-input-135-275af09cda96>:25: UserWarning: p-value floored: true value smaller than 0.001
  r_AD = anderson_ksamp([d1, d2])
<ipython-input-135-275af09cda96>:25: UserWarning: p-value floored: true value smaller than 0.001
  r_AD = anderson_ksamp([d1, d2])


KS test: v_grad (component 0): the two distributions are different at a 1.000% level (p-value = 0.0000000000%)
AD test: v_grad (component 0): the two distributions are different at a 1.000% level (p-value = 0.10000%)
Saving file to /priv/meggs3/u5708159/SAMI/figs/paper/hist_statistics_whole_sample.pdf
